# Imports
PhraseMatcher es per trobar les occurrencies dins el nostre training data aixi no hem de programar-ho nosaltres


In [17]:
import spacy
from spacy.matcher import PhraseMatcher
from pathlib import Path
import random
import urllib.request


In [18]:
# Carreguem el model que spAcy te en Espanyol per defecte
nlp = spacy.load('es_core_news_sm')

# variables globals
label = 'VEGGIE' # Nom de la nostre nova entitat
matcher = PhraseMatcher(nlp.vocab) # matcher de spacy per trobar ocurrencies

# offseter function
Serveix per trobar l'entitat a la linia en concret del nostre training data, trobem la posicio inicial i la posicio final

In [19]:
def offseter (label, doc, matchItem):
  o_one = len(str(doc[0:matchItem[1]])) + 1
  subdoc = doc[matchItem[1]:matchItem[2]]
  o_two = o_one + len(str(subdoc))
  return (o_one, o_two, label)

## Parsejem el dataset d'entranament

In [20]:
# Inicialitzem ner
if 'ner' not in nlp.pipe_names:
  ner = nlp.create_pipe('ner')
  nlp.add_pipe(ner)
else:
  ner = nlp.get_pipe('ner')

# Hi fotem la nostre nova entitat
ner.add_label(label)

In [21]:
# Afegim cada element de la nova entitat amb el label al matcher
for i in ['cebolla']:
  matcher.add(label, None, nlp(i))

In [37]:
file = urllib.request.urlopen("https://raw.githubusercontent.com/bocadoapp/nyora/master/entity-extractor/test.txt")
res = []
to_train_ents = []

for line in file:
    decoded_line = line.decode("utf-8")
    mnlp_line = nlp(decoded_line)
    matches = matcher(mnlp_line)
    res = [offseter(label, mnlp_line, x)
            for x
            in matches]
    to_train_ents.append((line, dict(entities=res)))

## Entrenem el robot

In [30]:
# Entreno el reconeixador
optimizer = nlp.begin_training()
other_pipes = [pipe
               for pipe
               in nlp.pipe_names
               if pipe != 'ner']

with nlp.disable_pipes(*other_pipes):
    for itn in range(20):
        losses = {}
        random.shuffle(to_train_ents)
        
        for item in to_train_ents:
            if item[0] != '' and len(item[1]['entities']) > 0:
                print(item, optimizer)
                nlp.update([item[0]], [item[1]], sgd=optimizer, drop=0.35, losses=losses)


(b'Estos componentes son los que sintetiza la planta para su propia defensa y las variedades moradas de cebolla tienen mayor cantidad\n', {'entities': [(101, 108, 'VEGGIE')]}) <thinc.neural.optimizers.Optimizer object at 0x121b076d0>


AttributeError: 'int' object has no attribute 'text'

## Testing model

In [8]:
# Carreguem text de prova
doc = nlp("""
Como guarnición, acompañamiento, "topping"... Esta cebolla caramelizada, una receta deliciosa y sencilla de elaborar, hará más delicioso si cabe todo aquello que toque. ¿Quieres probar una tortilla española distinta? Añádele esta cebolla confitada y déjate sorprender por el resultado. No te pierdas el paso a paso y aprende con RecetasGratis.net a cocinar la cebolla de esta forma, ¡te encantará!
""")

In [9]:
# Explorem les entitats trobades.
for ent in doc.ents:
    print(ent.label_, ent.text)

VEGGIE cebolla
VEGGIE cebolla


## Guardem el model entrenat

In [52]:
#nlp.meta['name'] = 'nyora'  # rename model
#nlp.to_disk('./model')
#print("Saved model to", './model')